forked from https://www.kaggle.com/kenseitrg/simple-fastai-exercise

In [1]:
!pip install pytorchcv

    100% |████████████████████████████████| 286kB 10.6MB/s 


In [2]:
!pip install fastai==1.0.47

    100% |████████████████████████████████| 215kB 33.0MB/s 
  Found existing installation: fastai 1.0.50.post1
    Uninstalling fastai-1.0.50.post1:
      Successfully uninstalled fastai-1.0.50.post1


In [3]:
import time
start = time.time()

In [4]:
import numpy as np 
import pandas as pd 
from pytorchcv.model_provider import get_model as ptcv_get_model

In [5]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch

In [6]:
data_folder = Path("../input")
#data_folder.ls()

In [7]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/sample_submission.csv")

In [8]:
test_img = ImageList.from_df(test_df, path=data_folder/'test', folder='test')
trfm = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)
train_img = (ImageList.from_df(train_df, path=data_folder/'train', folder='train')
        .split_by_rand_pct(0.01)
        .label_from_df()
        .add_test(test_img)
        .transform(trfm, size=128)
        .databunch(path='.', bs=64, device= torch.device('cuda:0'))
        .normalize(imagenet_stats)
       )

In [9]:
def md(f=None):
    mdl = ptcv_get_model('condensenet74_c4_g4', pretrained=True)
    mdl.features.final_pool = nn.AvgPool2d(kernel_size=7, stride=1, padding=3)
    return mdl

In [10]:
learn = cnn_learner(train_img, md, metrics=[error_rate, accuracy])

In [11]:
#learn.lr_find()
#learn.recorder.plot()

In [12]:
lr = 3.5e-02
learn.fit_one_cycle(5, slice(lr))

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.083160,0.177766,0.040000,0.960000,01:06
1,0.054930,0.036326,0.011429,0.988571,00:58
2,0.040563,0.048152,0.011429,0.988571,00:58
3,0.009462,0.001094,0.000000,1.000000,00:56
4,0.004515,0.001212,0.000000,1.000000,00:54


In [13]:
preds,_ = learn.get_preds(ds_type=DatasetType.Test)

In [14]:
test_df.has_cactus = preds.numpy()[:, 0]

In [15]:
test_df.to_csv('submission.csv', index=False)

In [16]:
end = time.time() 
print(end - start)

313.3328809738159
